In [27]:
import pandas as pd
import geopandas as gpd
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
# Use in Jupyter notebook: run 'jupyter nbextension enable --py --sys-prefix widgetsnbextension' in your terminal
#import ipyleaflet
# Use in Jupyter notebook: run 'jupyter nbextension enable --py --sys-prefix ipyleaflet' in your terminal
import json 
import altair as alt
from altair import Scale
alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

### Handle CSV data 
**Life expectancy and population data and projections**

In [28]:
gm_proj_df = pd.read_csv('data/exp_popn_proj_full.csv', usecols=range(1,6))

In [29]:
gm_proj_df.head()

,country,continent,year,lifeExp,pop
0,Afghanistan,Asia,1952,28.801,8425333.0
1,Afghanistan,Asia,1957,30.332,9240934.0
2,Afghanistan,Asia,1962,31.997,10267083.0
3,Afghanistan,Asia,1967,34.020,11537966.0
4,Afghanistan,Asia,1972,36.088,13079460.0


In [30]:
gm_proj_df = gm_proj_df.dropna()

### Handle Geo data

In [31]:
gpd_df = gpd.read_file('data/countries.geo.json')
        
data_geo = json.loads(gpd_df.to_json())

### Create plot and map

In [45]:
def create_plot(year):
    year_df = gm_proj_df[gm_proj_df.year == year]
    plot = alt.Chart(year_df, width=600, height=400).mark_circle(size=60).encode(
        x='pop',
        y=alt.Y('lifeExp', scale=Scale(zero=False)),
        color='continent',
        tooltip=['year', 'country', 'continent', 'lifeExp', 'pop']  
    ).interactive()
    
    map_background = alt.Chart(alt.Data(values=data_geo['features'])).mark_geoshape(
        fill='#967878',
        stroke='#979dc4'
    ).properties(
        width=600,
        height=400
    ).project('equirectangular')
    
    map_data = alt.Chart(alt.Data(values=data_geo['features'])).mark_geoshape().encode(
        color='lifeExp:Q'
    ).transform_lookup(
        lookup="properties.name",
        from_=alt.LookupData(year_df, 'country', ['lifeExp'])
    ).project(
        type='equirectangular'
    ).properties(
        width=600,
        height=400
    )
    
    chart = alt.vconcat(plot, map_background + map_data)
    return chart

In [46]:
year_range = sorted(set(gm_proj_df.year))
interact(create_plot, year=widgets.IntSlider(min=min(year_range), max=max(year_range), step=1, value=10));

interactive(children=(IntSlider(value=1950, description='year', max=2100, min=1950), Output()), _dom_classes=(…